<a href="https://colab.research.google.com/github/parabkrisna/arex-download/blob/main/make_metadata_GEO.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
file = open(r"/content/gds_result.txt")
text = file.read()
#print(text)

In [ ]:
import re
for line in text.splitlines():
  accessions = re.findall(r': GSE\d+', line)
  #print(accessions)

In [ ]:
import re
import csv

# Initialize an empty list to store non-empty accession IDs
accession_list = []

# Iterate over each line in the text
for line in text.splitlines():
    # Find all accession IDs in the line
    accessions = re.findall(r': GSE\d+', line)

    # Check if the list of accession IDs is not empty
    if accessions:
        # Add the accession IDs to the list
        accession_list.extend(accessions)

# Open a CSV file for writing
with open('accessions.csv', 'w', newline='') as csvfile:
    # Create a CSV writer object
    writer = csv.writer(csvfile)

    # Write the header row
    writer.writerow(['accessions'])

    # Write the accession IDs to the CSV file, one per column
    writer.writerows([[accession] for accession in accession_list])


In [ ]:
import re

for line in accession_list:
  gse = re.sub(r': GSE', 'GSE', line)
  #print(gse)


In [ ]:
gse_list = []

# Iterate over the accession IDs in the accession_list
for accession in accession_list:
    # Extract the GSE ID from the accession ID
    gse_id = re.sub(r': GSE', 'GSE', accession)

    # Add the GSE ID to the gse_list
    gse_list.append(gse_id)

# Open a CSV file for writing
with open('gse.csv', 'w', newline='') as csvfile:
    # Create a CSV writer object
    writer = csv.writer(csvfile)

    # Write the header row
    writer.writerow(['gse_id'])

    # Write the GSE IDs to the CSV file, one per column
    writer.writerows([[gse] for gse in gse_list])


In [ ]:
import pandas as pd

# Read the CSV file
misc = pd.read_csv("gse.csv", delimiter="\t")
misc = pd.DataFrame(misc)

# Example list of GSEIDs
gse_list = misc['gse_id'].tolist()

# Read the Excel file
obesity_geo_curation_19may2023_originaldoc = pd.read_excel("obesity_geo_curation_19may2023(originaldoc).xlsx", sheet_name="Series_all")

# Extract relevant columns
GEO_accession_data = obesity_geo_curation_19may2023_originaldoc[['GEO_accession']]

# Common data
common = misc[misc['gse_id'].isin(GEO_accession_data['GEO_accession'])]

# Uncommon data
update_list = misc[~misc['gse_id'].isin(GEO_accession_data['GEO_accession'])]

# Convert uncommon data to DataFrame
updated_df = pd.DataFrame(update_list)


In [ ]:
file = open(r"/content/gds_result.txt")
text = file.readlines()
print(text)

['\n', '1. Dissecting the Genetics of the Human Transcriptome identifies novel trait-related trans-eQTLs and corroborates the regulatory relevance of non-protein coding loci\n', '(Submitter supplied) Genetics of gene expression (eQTLs or expression QTLs) has proved an indispensable tool for understanding biological pathways and pathomechanisms of trait associated SNPs. However, power of most genome-wide eQTL studies is still limited. We performed a large eQTL study in peripheral blood mononuclear cells of 2,112 individuals increasing the power to detect trans-effects genome-wide. Going beyond univariate SNP-transcript associations, we analyse relations of eQTLs to biological pathways, polygenetic effects of expression regulation, trans-clusters, and enrichment of co-localised functional elements. more...\n', 'Organism:\tHomo sapiens\n', 'Type:\t\tExpression profiling by array\n', 'Platform: GPL10558 2112 Samples\n', 'FTP download: GEO (TXT) ftp://ftp.ncbi.nlm.nih.gov/geo/series/GSE65nn

In [ ]:
import pandas as pd

def split_paragraphs(passages):
    paragraphs = []
    current_paragraph = []

    for line in passages:
        if line.strip() == "":
            if current_paragraph:
                paragraphs.append(current_paragraph)
                current_paragraph = []
        else:
            current_paragraph.append(line.strip())

    if current_paragraph:
        paragraphs.append(current_paragraph)

    return paragraphs

paragraphs = split_paragraphs(text)

org = []
ftp = []
acc = []
platf = []
type_e = []
sum =[]
title =[]

def lcontains(needle_s, haystack_l):
    try: return [i for i in haystack_l if needle_s in i]
    except IndexError: return None

for i, paragraph in enumerate(paragraphs):
    sentences = [sentence for sentence in paragraph if sentence != ""]
    sum.extend(paragraphs[i][1:2])
    title.extend(paragraphs[i][:1])
    org.extend(lcontains("Organism", paragraphs[i]))
    ftp.extend(lcontains("FTP", paragraphs[i]))
    acc.extend(lcontains("Accession", paragraphs[i]))
    type_e.extend(lcontains("Type:", paragraphs[i]))
    platf.extend(lcontains("Platf", paragraphs[i]))

org_df = pd.DataFrame(org, columns=["Organism"])
ftp_df = pd.DataFrame(ftp, columns=["FTP"])
acc_df = pd.DataFrame(acc, columns=["Accession"])
type_e_df = pd.DataFrame(type_e, columns=["Type"])
platf_df = pd.DataFrame(platf, columns=["Platf"])
sum_df = pd.DataFrame(sum, columns=["Summary"])
title_df = pd.DataFrame(title, columns=["Title"])

meta_data = pd.concat([ acc_df, title_df, sum_df, type_e_df, platf_df, org_df, ftp_df], axis=1)


In [ ]:
paragraphs[i][1:2]

['(Submitter supplied) Expression profiling of androgen and insulin pathway regulating genes unveils SOS 1 as candidate gene for idiopathic hirsutism Background: Idiopathic Hirsutism (IH), defined as the presence of terminal hairs in females in a male-like pattern, affecting about 5% -15% of women, may result from the interaction between the androgen level and the sensitivity of the hair follicle to the androgen. The pathophysiology of IH is presumed to be linked to increased SRD5A activity, probably both isoenzyme types (SRD5A1 and SRD5A2) and generally related to an alteration in androgen receptor (AR) function. more...']

In [ ]:
meta_data = meta_data.dropna()
meta_data["GEO_Accession"] = meta_data['Accession'].str.extract(r'(GSE\d+)')
meta_data["Exp Organism"] = meta_data['Organism'].map(lambda x: x.lstrip('Organism:\t'))
meta_data["Experiment Type"] = meta_data['Type'].map(lambda x: x.lstrip('Type:\t\t'))
meta_data["Platform"] = meta_data['Platf'].str.extract(r'(GPL\d+)')
meta_data["Sample count"] = meta_data['Platf'].str.extract(r'(\s\d+)')
meta_data["File Format"] = meta_data['FTP'].map(lambda x: x.lstrip('FTP download: GEO').rstrip(""))

In [ ]:
drop_list = ["Organism","FTP","Accession","Type","Platf"]
meta_data.drop(drop_list, axis=1, inplace=True)

In [ ]:
meta_data

,Title,Summary,GEO_Accession,Exp Organism,Experiment Type,Platform,Sample count,File Format
0,1. Dissecting the Genetics of the Human Transc...,(Submitter supplied) Genetics of gene expressi...,GSE65907,Homo sapiens,Expression profiling by array,GPL10558,2112,(TXT) ftp://ftp.ncbi.nlm.nih.gov/geo/series/GS...
1,2. Persistent Variations of Blood DNA Methylat...,(Submitter supplied) Background: It is well-es...,GSE169156,Homo sapiens,Methylation profiling by genome tiling array,GPL23976,2052,"(IDAT, TXT) ftp://ftp.ncbi.nlm.nih.gov/geo/ser..."
2,3. Population genetic properties of differenti...,(Submitter supplied) Copy number variants (CNV...,GSE26450,Homo sapiens,Genome variation profiling by genome tiling array,GPL11386,1434,(TXT) ftp://ftp.ncbi.nlm.nih.gov/geo/series/GS...
3,4. DNA methylation analysis of human periphera...,(Submitter supplied) Markers of biological age...,GSE147740,Homo sapiens,Methylation profiling by genome tiling array,GPL21145,1129,"(IDAT, TXT) ftp://ftp.ncbi.nlm.nih.gov/geo/ser..."
4,5. Liver transcriptome of statin-treated patients,(Submitter supplied) Background: Clinical data...,GSE130991,Homo sapiens,Expression profiling by array,GPL20265,910,(CEL) ftp://ftp.ncbi.nlm.nih.gov/geo/series/GS...
...,...,...,...,...,...,...,...,...
755,756. THE EFFECTS OF ALCOHOLISM ON THE HUMAN BA...,(Submitter supplied) Alcohol affects gene expr...,GSE18212,Homo sapiens,Expression profiling by array,GPL11154,2,(TXT) ftp://ftp.ncbi.nlm.nih.gov/geo/series/GS...
756,757. Modeling maternal cholesterol exposure re...,(Submitter supplied) Maternal obesity raises t...,GSE206587,Homo sapiens,Expression profiling by high throughput sequen...,GPL13607,2,(H5) ftp://ftp.ncbi.nlm.nih.gov/geo/series/GSE...
757,758. Integration of human adipocyte chromosoma...,(Submitter supplied) We perform promoter Captu...,GSE110619,Homo sapiens,Other,GPL570,2,(TXT) ftp://ftp.ncbi.nlm.nih.gov/geo/series/GS...
758,759. Investigation of a patient with a partial...,(Submitter supplied) A female patient with a p...,GSE12741,Homo sapiens,Genome variation profiling by genome tiling array,GPL11154,2,(TXT) ftp://ftp.ncbi.nlm.nih.gov/geo/series/GS...


In [ ]:
meta_data.to_csv("GEO_curation_16April2024_doc.csv")